In this notebook, features are engeneered such that they are more suitable for machine learning classification algorithms. The main points of this notebook are as follows:
1. Any date-time feature in users table are decomposed into three new features, namely year, month, day of month (see notebook 'EDA_MissingValue&Visualization' for details why this should be done).

2. Missing values are imputed using the mean variable substitution with indicator variable (see notebook 'EDA_MissingValue&Visualization' for details for why this should be done).

3. All categorical features are encoded using one hot encoding.

4. New features extracted from web session table are merged into users table.

5. Feature scaling is applied to non-binary features (e.g., indicator features are binary features).

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#from xgboost.sklearn import XGBClassifier

np.random.seed(0)

from sklearn.ensemble import RandomForestClassifier
#from sklearn.cross_validation import cross_val_score
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
#import sklearn
#import sklearn.grid_search
#from sklearn.cross_validation import train_test_split
#from sklearn.cross_validation import KFold
import sklearn.preprocessing as preprocessing

import math

#from NDCG_score_func import ndcg_scorer
from __future__ import division

Loading data

First, Let's loading users table for train and test users. And do some additional necessary operations.

In [4]:
df_train = pd.read_csv('train_users_2.csv')
df_test = pd.read_csv('test_users.csv')


labels = df_train['country_destination'].values
df_train = df_train.drop(['country_destination'], axis=1) #drop the dependent/target variable

id_test = df_test['id']
piv_train = df_train.shape[0]  #

Creating a DataFrame with train+test data

In [5]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
## Removing date_first_booking
df_all = df_all.drop(['date_first_booking'], axis=1)
#set id as index 
df_all.set_index('id',inplace=True)

### 1. About date-time features

Previously, it is seen that booking behaves very differently depending on the year and month on which they registered. Therefore, feature decomposition is performed with date-time features, namely each feature of type datetime is split into three new ones: year, month, day of month.

#### date_account_created

In [6]:
##lambda x: map(int, x.split('-'))：converting string 'year-month-day'  to [year,month,day] (int type)c
dac = np.vstack(df_all.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
df_all['dac_year'] = dac[:,0]
df_all['dac_month'] = dac[:,1]
df_all['dac_day'] = dac[:,2]
df_all = df_all.drop(['date_account_created'], axis=1)

In [7]:
df_all.head()

,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,dac_year,dac_month,dac_day
id,,,,,,,,,,,,,,,
gxn3p5htnn,20090319043255,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,6,28
820tgsjxq7,20090523174809,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2011,5,25
4ft3gnwmtx,20090609231247,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2010,9,28
bjjt8pjhuk,20091031060129,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2011,12,5
87mebub9p4,20091208061105,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,9,14


#### timestamp_first_active

In [9]:
#dac = np.vstack(df_all.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
tfa = np.vstack(df_all.timestamp_first_active.astype(str).apply(lambda x: list(map(int,[x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
df_all['tfa_year'] = tfa[:,0]
df_all['tfa_month'] = tfa[:,1]
df_all['tfa_day'] = tfa[:,2]
df_all = df_all.drop(['timestamp_first_active'], axis=1)

In [10]:
df_all.head()

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day
id,,,,,,,,,,,,,,,,,
gxn3p5htnn,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,6,28,2009,3,19
820tgsjxq7,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2011,5,25,2009,5,23
4ft3gnwmtx,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2010,9,28,2009,6,9
bjjt8pjhuk,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2011,12,5,2009,10,31
87mebub9p4,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,9,14,2009,12,8


### 2. Missing values imputation

In this part, only the missing values for age variable are handled. To be more specific, the ages within the range 16 years old to 95 years old are considered to be valid. All other entries for age variable beyond this range are considered to be invalid. First the mean is computed by averaging all valid ages. The entries of all missing values and the invalid entries are filled by this computed mean. Moreover, two  indicator variables are used to indicate the status of 'missing' and 'invalid' respectively.

Missing values for features extracted from websession table are addressed in section "4. Merging 'User' and new feature tables extracted from 'Session' table". Missing values for categorical features are handled by one hot encoding scheme (see Section '3. One-hot-encoding features' for details).

In [11]:
#Mean of the normal data
mean_age = df_train[(df_train.age<95) & (df_train.age>16)].age.mean()
av = df_all.age.values

#1. fill all nan values by the mean of normal data,use a dummy variable to denote it
df_all['age_nan'] = np.where(df_all.age.isnull(), 1, 0)
df_all['age'] = np.where(np.isnan(av), mean_age, av)

#2. some abnormal data, e.g., 2014, is filled by the mean, use a dummy variable to denote it
df_all['age_abno'] = np.where(np.logical_or(av<16, av>95), 1, 0)
df_all['age'] = np.where(np.logical_or(av<16, av>95), mean_age, df_all['age'])

### 3. One-hot-encoding features

All categorical features in the users table are encoded. These features are: 'gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser'.

In [12]:
# ohe_feats = ['first_affiliate_tracked']
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider',\
             'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in ohe_feats:
    df_all_dummy = pd.get_dummies(df_all[f], prefix=f,dummy_na=True)
    df_all = df_all.drop([f], axis=1)
    df_all = pd.concat((df_all, df_all_dummy), axis=1)

### 4. Merging 'User' and new feature tables extracted from 'Session' table：

In [15]:
#loading tables
df_action_t = pd.read_csv('df_action_t.csv')
df_action_t.set_index('user_id',inplace=True)

df_action = pd.read_csv('df_action.csv')
df_action.set_index('user_id',inplace=True)

df_action_d = pd.read_csv('df_action_d.csv')
df_action_d.set_index('user_id',inplace=True)

# Merging
df_all = pd.merge(df_all,df_action_t, left_index = True, right_index = True, how = 'left')
df_all = pd.merge(df_all,df_action, left_index = True, right_index = True, how = 'left')
df_all = pd.merge(df_all,df_action_d, left_index = True, right_index = True, how = 'left')

It is noted that the web session table only contains data for users registerd after Jan. 1st, 2014. Therefore, for those users registered after Jan. 1st, 2014, a indicator variable is needed to indicate the missingness status of web session data. However, recall that not all the web session behaviors are included in the extrated new features (see notebook 'Feature_engineering__session_Feature_extraction' for details), e.g., not all values for 'action_detail' are included. As a result, a separate indicator variable is needed for each of the original variables 'action', 'action_type','action_detail' in the session table.

In [18]:
#Computing mean for all extracted features from session table
session_mean =df_all.iloc[:,172:].mean()
df_all.iloc[:,172:] = df_all.iloc[:,172:].fillna(session_mean)

In [19]:
#adding indicator variables
#indicator list
indic = df_all.index.isin(df_action_t.index)
df_all['action_t_indic'] = np.where(indic, 1, 0)

indic = df_all.index.isin(df_action.index)
df_all['action_indic'] = np.where(indic, 1, 0)

indic = df_all.index.isin(df_action_d.index)
df_all['action_d_indic'] = np.where(indic, 1, 0)

### 5. Feature scaling

In the following, zero-mean-unit-variace scaling and min-max scaling are used to scale the datas. 

#### Date-time features

Characteristic: this type of features has no outlier problem, they includes: features extracted from date_account_created and timestamp_first_active. 

Solution: min-max scaling (range (-1,1)), because datas are evenly distributted (not normally distributed).

In [21]:
#take all relavant data columns and apply function to them together. Perhaps numpy arry is needed...
df_date = df_all[['dac_year','dac_month','dac_day','tfa_year','tfa_month','tfa_day']].copy()
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1),)
df_date = pd.DataFrame(data =  min_max_scaler.fit_transform(df_date), index=df_date.index, columns=df_date.columns)  
df_date.head()

,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day
id,,,,,,
gxn3p5htnn,-1.0,-0.090909,0.800000,-1,-0.636364,0.200000
820tgsjxq7,-0.5,-0.272727,0.600000,-1,-0.272727,0.466667
4ft3gnwmtx,-1.0,0.454545,0.800000,-1,-0.090909,-0.466667
bjjt8pjhuk,-0.5,1.000000,-0.733333,-1,0.636364,1.000000
87mebub9p4,-1.0,0.454545,-0.133333,-1,1.000000,-0.533333


In [22]:
df_all[['dac_year','dac_month','dac_day','tfa_year','tfa_month','tfa_day']]=df_date

#### Numerical features
This type of features includes:
- age (outliers already addressed): normally distributed, standard scaling (centering and scaling) is used
- all features generated from session tables except for indicator variables (outliers not addressed): min-max scaling used here because the range is garenteed compared to some other approaches, such as median-interquatile sclaling or maxabs_scale scaling. Actually, zero-mean-unit-variace scaling was also tried, and the performance is very similar to min-max scaling.

###### age

In [23]:
df_test=df_all['age'].copy()
df_test=pd.Series(data=preprocessing.scale(df_test),index=df_test.index)
df_all['age'] = df_test

###### Aggregation variables

In [24]:
df_tem = df_all.iloc[:,172:-3].copy()

# standard scaling
# df_tem = pd.DataFrame(data = preprocessing.scale(df_tem),index=df_tem.index,columns=df_tem.columns)

#min-max scaling
df_tem = pd.DataFrame(data = preprocessing.minmax_scale(df_tem,feature_range=(-1,1)),index=df_tem.index,columns=df_tem.columns)
# maxabs_scale scaling
# df_tem = pd.DataFrame(data = preprocessing.maxabs_scale(df_tem),index=df_tem.index,columns=df_tem.columns)

# #median-interquatile sclaling...
# df_tem = pd.DataFrame(data = preprocessing.minmax_scale(df_tem),index=df_tem.index,columns=df_tem.columns)
df_tem.head()

,-unknown-_x,Missing_actt,booking_request,booking_response,click,data,message_post_x,modify,partner_callback,submit,...,message_thread,p3,similar_listings_y,unavailable_dates,update_listing,user_profile,user_social_connections,user_wishlists,view_search_results,wishlist_content_update
id,,,,,,,,,,,,,,,,,,,,,
gxn3p5htnn,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988
820tgsjxq7,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988
4ft3gnwmtx,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988
bjjt8pjhuk,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988
87mebub9p4,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988


In [25]:
df_all.iloc[:,172:-3] = df_tem
df_all.iloc[:,172:-3].head()

,-unknown-_x,Missing_actt,booking_request,booking_response,click,data,message_post_x,modify,partner_callback,submit,...,message_thread,p3,similar_listings_y,unavailable_dates,update_listing,user_profile,user_social_connections,user_wishlists,view_search_results,wishlist_content_update
id,,,,,,,,,,,,,,,,,,,,,
gxn3p5htnn,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988
820tgsjxq7,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988
4ft3gnwmtx,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988
bjjt8pjhuk,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988
87mebub9p4,-0.994233,-0.979433,-0.960434,-0.999941,-0.970236,-0.964052,-0.997186,-0.999328,-0.984374,-0.973157,...,-0.993853,-0.984217,-0.983362,-0.993298,-0.98755,-0.978546,-0.985816,-0.991287,-0.973302,-0.979988


#### Indicator variables (binary variables)

This type of features include
- Features resulted from one-hot-encoding
- All indicator features

Solution: leave them as is

### Splitting data set

Finally, the whole data set is split into train and test data set for the convience of later processing.

In [ ]:
#dataFrame to numpy arrays
vals = df_all.values

#training set
X = vals[:piv_train]
#transform labels into integers
le = LabelEncoder()
y = le.fit_transform(labels) #is this  integer encoding a must-do or optional?

#shuffling
y = y.reshape((len(y),1))
Xy = np.concatenate((X,y),axis = 1)
np.random.shuffle(Xy)
X = Xy[:,:(X.shape[1])]
y = Xy[:,X.shape[1]]


#test set
X_test = vals[piv_train:]

##### Save X,y and X_test
np.save('X.npy', X) 
np.save('y.npy', y) 
np.save('X_test.npy', X_test) 
df_all.to_csv('df_all.csv')

Datas are now ready to be processed by machine learning classification algorithms. let's see the performance!